## Data synthesis: new dataframe for model eval and deployed with recommmender app

Dataset synthesis: After BoW represetnation is constructed, we combine the script dataset with IMDb the dataset to assist with model evaluation and eventually to be deployed with recommender app.

In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
# import (cleaned) movie script data
path_to_csv = (
    "data_cleaning_and_synthesis_out\springfield_movie_scripts_2023_01_13_clean.csv"
)
df = pd.read_csv(path_to_csv, index_col=[0])
df = df[["movie_title", "movie_year", "imdb_id", "imdb_link", "tmdb_poster_link"]]

# load in any changes to df from data_preprocessing_eda.ipynb notebook
# TODO - add functionality to give option to drop movie scripts after datapreprocessing

# load in IMDb title basics and title ratings datasets
path_to_tsv = "..\\database_IMDb\\2023_02_12_IMDb_dataset\\titleBasics.tsv"
df_imdb_basics = pd.read_csv(path_to_tsv, sep="\t")
df_imdb_basics = df_imdb_basics[["tconst", "genres", "isAdult"]]

path_to_tsv = "..\\database_IMDb\\2023_02_12_IMDb_dataset\\titleRatings.tsv"
df_imdb_ratings = pd.read_csv(path_to_tsv, sep="\t")

# join the two datasets on tconst
df_imdb = df_imdb_basics.join(
    other=df_imdb_ratings.set_index("tconst"), on="tconst", how="left"
)

# finally join springfield and imdb datasets
df = df.join(df_imdb.set_index("tconst"), on="imdb_id", how="left")

# rename and reorder the columns
df.columns = [
    "movie_title",
    "movie_year",
    "imdb_id",
    "imdb_link",
    "tmdb_poster_link",
    "genre",
    "is_adult",
    "average_rating",
    "num_votes",
]
df = df[
    [
        "movie_title",
        "movie_year",
        "genre",
        "average_rating",
        "num_votes",
        "is_adult",
        "imdb_id",
        "imdb_link",
        "tmdb_poster_link",
    ]
]

# # drop movies that did not have any words after constructing bag of words
# df = df.drop(jmovie_drop, axis=0)

# add year to movie titles with duplicate names
movie_title_counts = df["movie_title"].value_counts()
movie_dups = movie_title_counts[movie_title_counts > 1]
list_of_dups = movie_dups.index.tolist()

df["movie_title"] = [
    jmovie_title + ", " + str(df["movie_year"].iloc[j])
    if jmovie_title in list_of_dups
    else jmovie_title
    for j, jmovie_title in enumerate(df["movie_title"])
]

# clean up explicits in the movie titles
df["movie_title"] = [
    jmovie.replace("uck", "***") if "fuck" in jmovie.lower() else jmovie
    for jmovie in df["movie_title"]
]
df["movie_title"] = [
    jmovie.replace("itch", "****") if "bitch" in jmovie.lower() else jmovie
    for jmovie in df["movie_title"]
]
df["movie_title"] = [
    jmovie.replace("hit", "***") if "shit " in jmovie.lower() else jmovie
    for jmovie in df["movie_title"]
]

# save new dataframe for uploading to spaces
df.to_csv("data_preprocessing_eda_out\\df_spaces_upload.csv")

C:\Users\Nick\AppData\Local\Temp\ipykernel_26844\2606149448.py:10: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_imdb_basics = pd.read_csv(path_to_tsv, sep="\t")
